# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверяем гипотезы:
1. Возврат кредита в срок зависит от наличия детей.
2. Возврат кредита в срок зависит от семейного положения заёмщика.
3. Между уровнем дохода и возвратом кредита в срок существует зависимость.
4. Возврат кредита в срок различно зависит от цели получения кредита.

**Ход исследования**

Данные о платежеспособности получены из файла `/datasets/data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. Потребуется оценить наличие ошибок в данных и по возможности исправить самые критичные из них.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

In [1]:
import pandas as pd

from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')

from pymystem3 import Mystem
m = Mystem() 

import collections
from collections import Counter

try:
    df = pd.read_csv('C:/Users/datasets/data.csv')  # Локальный путь
except:
    df = pd.read_csv('/datasets/data.csv')  # Серверный путь
    
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Типы данных в столбцах — `float64`, `int64`,`object`.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Количество значений в столбцах различается. В столбцах `days_employed` и `total_income`данных есть пропущенные значения.

In [3]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Выводы**

В каждой строке таблицы — данные о заёмщике. Часть колонок содержит данные о заемщике:семейное положение, в том числе количестве детей заемщика, возрасте, уровне образования, пол заемщика. Часть колонок содержит информацию о трудоустройстве заемщика: стаж, тип занятости и ежемесячных доход. Часть колонок отражает информацию о кредите: целях получения кредита и наличии задолженностей.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в некоторых колонках значения отличаются по регистру, цели кредита имеют различные вариации для описания подобных целей. Общий стаж в днях частично представлен отрицательными значениями, частично нереалистично большими. При этом встречается отрицательное количество детей и аномальное 20 в столбце `children`, данные к тому же представлены в неудобном формате.

## Предобработка данных

### Обработка пропусков

Проверим, в каких столбцах встречаются пропущенные значения.

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В таблице имеется 2174 строки с пропущенными значениями в столбцах `days_employed` и `total_income`. То есть у 10% датасета отсутствуют значения в этих столбцах. Пропуски в `days_employed` не повлияют на проверку гипотез, но столбец `total_income` важен для исследования, а отсутствующие значения могут сильно повлиять на конечный результат. Поэтому заменим пропуски на медианные значения для каждого типа занятости. Поскольку количество пропусков одинаково именно в этих столбцах, это наталкивает на мысль, что в датасете просто присутствуют люди, у которых нет стажа и, соответственно, нет дохода. Либо при выгрузке конкретно этих значений произошел сбой.

Заменим пропуски на медианные значения для каждого типа занятости.

In [5]:
income_types = df['income_type'].unique()

for type in income_types:
    median = df.loc[df['income_type'] == type, 'total_income'].median()
    df.loc[(df['total_income'].isna()) & (df['income_type'] == type), 'total_income'] = median

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>
    
Хорошая  замена пропущенных значений на медиану.</div>

Займемся трудовым стажем. Встречаются отрицательные числа, заменим их на те же числа, но по модулю.

In [6]:
df['days_employed'] = abs(df['days_employed'])

Для типа занятости пенсионер и безработный наблюдаются нереалистичные значения стажа, вероятно, вычисленного в часах, а не днях. Заменим эти значения.

In [7]:
df.loc[df['days_employed'] > df['dob_years'] * 365, 'days_employed'] = df.loc[df['days_employed'] > df['dob_years'] * 365, 'days_employed'] / 24
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверим, остались ли в таблице пропуски.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Выводы**

Пропуски присутствуют в столбцах `days_employed` и `total_income` в одинаковых количествах, следовательно это могут быть люди, у которых пока нет стажа, следовательно нет и дохода, также возможен сбой при выгрузке значений конкретно для этих строк. Не все пропущенные значения влияют на исследование. Так в `days_employed` пропуски не столь важны для доказательства гипотез. Поэтому принято решение оставить значения NaN.

Но пропуски в `total_income` могут помешать при выяснении связи между уровнем дохода и возвратом кредита в срок. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Поэтому произведена замена пропусков на медианные значения для каждого типа занятости.

### Замена типа данных

Столбцы `total_income` и `children` представлены типом float64. Для удобства восприятия и проведения математических операций, заменим вещественный тип на целочисленный методом astype().

In [9]:
df['total_income'] = df['total_income'].astype('int')
df['children'] = df['children'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Также обработаем выявленные аномальные значения -1 и 20 в столбце с количеством детей.

In [10]:
df['children'] = abs(df['children'])
ch_median = df.loc[df.loc[:, 'children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, ch_median)

**Выводы**

Данные в таблице представлены в удобных для анализа типах.

### Обработка дубликатов

Избавимся от неявных дубликатов в столбце `education`.

In [11]:
sorted(df['education'].unique())

['ВЫСШЕЕ',
 'Высшее',
 'НАЧАЛЬНОЕ',
 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
 'Начальное',
 'Неоконченное высшее',
 'СРЕДНЕЕ',
 'Среднее',
 'УЧЕНАЯ СТЕПЕНЬ',
 'Ученая степень',
 'высшее',
 'начальное',
 'неоконченное высшее',
 'среднее',
 'ученая степень']

Видим, что в столбце с образованием дубликаты возникают из-за различного регистра, скорее всего это возникло при заполнении данных, поскольку люди, заполнявшие эти графы, использовали различные регистры. Исправим это.

In [12]:
df['education'] = df['education'].str.lower()

Проверим результат.

In [13]:
df['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

Подсчитаем явные дубликаты в таблице.

In [14]:
df.duplicated().sum()

71

Удалим явные дубликаты методом drop_duplicates().

In [15]:
df = df.drop_duplicates().reset_index(drop=True)

Проверим наличие явных дубликатов ещё раз.

In [16]:
df.duplicated().sum()

0

**Выводы**

Явные дубликаты удалены методом drop_duplicates(), значения в столбце с образованием приведены к единому регистру. Дубликаты могли возникнуть из-за ошибки ввода. Некоторые данные могли быть введены несколько раз, ещё и с разным регистром.

### Лемматизация

Оценим все уникальные виды переменных из столбца `purpose`.

In [17]:
sorted(df['purpose'].unique())

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

Для каждого элемента найдем основу.

In [18]:
df['purpose_lemmatize']=df['purpose'].apply(m.lemmatize)

In [19]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmatize
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]"
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


In [20]:
words = str(df['purpose_lemmatize']).split()
Counter(words).most_common()

[(',', 14),
 ('\\n]', 10),
 ('автомобиль,', 4),
 ('жилье,', 3),
 ('[покупка,', 2),
 ('с,', 2),
 ('[на,', 2),
 ('покупка,', 2),
 ('0', 1),
 ('1', 1),
 ('[приобретение,', 1),
 ('2', 1),
 ('3', 1),
 ('[дополнительный,', 1),
 ('образование,', 1),
 ('4', 1),
 ('[сыграть,', 1),
 ('свадьба,', 1),
 ('...', 1),
 ('21449', 1),
 ('[операция,', 1),
 ('21450', 1),
 ('[сделка,', 1),
 ('21451', 1),
 ('[недвижимость,', 1),
 ('21452', 1),
 ('свой,', 1),
 ('21453', 1),
 ('Name:', 1),
 ('purpose_lemmatize,', 1),
 ('Length:', 1),
 ('21454,', 1),
 ('dtype:', 1),
 ('object', 1)]

**Выводы**

В столбце с целью много вариаций записи одних и тех же целей. Исходя из того, какие значения встречаются чаще всего, можно выделить основные 4: автомобиль, недвижимость, свадьба и образование. Благодаря столбцу с леммами, можно категоризировать данные.

### Категоризация данных

Видим, что в столбце с целью кредита много похожим по сути записей, но в различном виде. Всего можно выделить четыре категории. Напишем функцию, которая будет возвращать нужную категорию вместо лемматизированной переменной.

In [21]:
def purpose(row):
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'свадьба' in row:
        return 'свадьба'
    if 'образование' in row:
        return 'образование' 
    if 'недвижимость' or 'жилье' in row:
        return 'недвижимость'

df['purpose_category'] = df['purpose_lemmatize'].apply(purpose)
df['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

Теперь все данные удобно поделены на 4 категории в зависимости от цели кредита.

Теперь проведем категоризацию по столбцу с доходами и распределим заемщиков по 4 категориям в зависимости от уровня дохода с шагом в 43.788.

In [22]:
df['total_income'].describe()

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

Видим, что минимальное значение дохода составляет 20.667, максимальный 2.265.604, при этом по показателям видно, что людей, близких по доходу к максимальному достаточно мало, поэтому как очень высокий доход будем рассматривать 195.820 и выше. Разницу между низким и очень высоким поделим на 4, чтобы получить четыре категории низкого, среднего, высокого и очень высокого дохода.

In [23]:
def total_income_category(total_income):
    if total_income >= 20666 and total_income <= 64454:
        return 'низкий'
    if total_income > 64454 and total_income <= 108242:
        return 'средний'
    if total_income > 108242 and total_income <= 152030:
        return 'выше среднего'
    if total_income > 152030 and total_income <= 195820:
        return 'высокий'
    if total_income > 195820:
        return 'очень высокий'
    
    
df['total_income_category'] = df['total_income'].apply(total_income_category)

Также требуется категоризация по возрасту, разобьет заемщиков по двум категориям: до 30 лет, от 30.

In [24]:
def age(dob_years):
    if dob_years <=30:
        return 'молодые'
    if dob_years >=31:
        return 'взрослые'
df['age_category'] = df['dob_years'].apply(age)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmatize,purpose_category,total_income_category,age_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,очень высокий,взрослые
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,выше среднего,взрослые
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,выше среднего,взрослые
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,очень высокий,взрослые
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,высокий,взрослые
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,очень высокий,молодые
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,очень высокий,взрослые
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,выше среднего,взрослые
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,средний,взрослые
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,выше среднего,взрослые


**Выводы**

Методом лемматизации в таблицу добавлен столбец, содержащий результаты этой операции, который впредь можно использовать для категоризации целей кредита. Выбраны наиболее часто встречаюшиеся категории для целей кредита, так, ими оказались: образование, автомобиль, недвижимость и свадьба. В соответствии с этим проведена категоризация целей кредита. Доход также потребуется нам для проверки гипотез, поэтому было решено разбить заемщиков на несколько категорий, исходя из минимальных и максимальных значений в столбце дохода, а также их разницы. Выбраны категории для проведения проверки гипотез. Так, принято решение поделить заемщиков в зависимости от возраста на две категории: взрослые и молодые, это люди после 30 и до соответственно, так как взятие кредитов часто различно для разных возрастных категорий и правильнее будет учитывать этот факт. 

## Проверка гипотез

**3.1 Анализ зависимости между наличием детей и возвратом кредита**

При анализе были выявлены отрицательные значения количества детей, что является опечаткой. Предварительно заменим значения в столбце их модулями. Также попадаются строки со значением 20 детей, поскольку нет никаких промежуточных вариантов, а строк со значением 20 достаточно много, можно предположить, что это также опечатка. Не совсем понятно каким именно образом эта опечатка появилась, поэтому заменим значение количества детей в этих строках на медианное.

In [25]:
child = pd.pivot_table(df, index = ['children'], values='debt', aggfunc = ('mean', lambda X:X.count()))

def perc(x):
    return "{0:.2%}".format(x)

child['mean'] = child['mean'].apply(perc)
child = child.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее"})
child.sort_values(by='Количество записей', ascending=False)



,Количество записей,Среднее
children,,
0,14167.0,7.56%
1,4855.0,9.17%
2,2052.0,9.45%
3,330.0,8.18%
4,41.0,9.76%
5,9.0,0.00%


**Вывод**

По результатам видно, что бездетные люди имеют меньше всего задолженностей по кредиту, что может быть связано с тем, что в семьях без детей меньше незапланированных трат, что позволяет вовремя выплачивать заем. При этом в семьях, с одним/двумя/четырьмя детьми, процент заемщиков, не выплативших кредит в срок, увеличивается в сравнение с бездетными на 1.6-2.2 процентных пункта. Однако, число не вернувших кредит в срок среди заемщиков без детей больше, чем у заемщиков с тремя детьми, однако выборка с большим количеством детей достаточно мала и не позволяет с точностью утверждать, что при соразмерном количестве данных, картина выглядела бы также.

**3.2 Анализ зависимости между семейным положением и возвратом кредита в срок**

Оценим процент невернувших в срок кредит, в зависимости от семейного положения.

In [26]:
df['family_status'] = df['family_status'].str.lower()

f_status = pd.pivot_table(df, index = ['family_status'], values='debt', aggfunc = ('mean', lambda X:X.count()))

f_status['mean'] = f_status['mean'].apply(perc)
f_status = f_status.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее"})
f_status.sort_values(by='Количество записей', ascending=False)

,Количество записей,Среднее
family_status,,
женат / замужем,12339.0,7.55%
гражданский брак,4151.0,9.35%
не женат / не замужем,2810.0,9.75%
в разводе,1195.0,7.11%
вдовец / вдова,959.0,6.57%


**Вывод**

Видим, что самый высокий процент невозврата у категории заемщиков, не состоящих ни в каких отношениях. Вероятно, что им требуются кредиты на какие-либо личные цели и из-за отсутствия семьи и вероятности, что им потребуется кредит в будущем, они не сильно следят за сроками возврата и своей кредитной историей. При этом наблюдается на 2 процентных пункта меньше людей, не погасивших кредит вовремя среди заемщиков в официально оформленном браке. Возможно, это связано с тем, что в семьях доходы обычно больше, чем у людей не в браке, и к распределению бюджета, в том числе на погашение кредита, они подходят ответственнее, сохраняют положительную кредитную историю. У заемщиков вдова/вдовец и в разводе наблюдается тенденция выплачивать кредит в срок чаще, чем у заемщиков с другим семейным положением.

**3.3 Анализ зависимости между уровнем дохода и возвратом кредита в срок**

In [27]:
inc = pd.pivot_table(df, index = 'total_income_category', values='debt', aggfunc = ('mean', lambda X:X.count()))

inc['mean'] = inc['mean'].apply(perc)
inc = inc.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее по задолжностям"})
inc.sort_values(by='Количество записей', ascending=False)

,Количество записей,Среднее по задолжностям
total_income_category,,
выше среднего,6517.0,8.78%
очень высокий,5364.0,7.14%
средний,4379.0,8.31%
высокий,4127.0,8.50%
низкий,1067.0,6.65%


**Вывод**

Судя по получившимся данным, чаще всего в срок кредит выплачивают люди либо с очень высоким доходом, либо с низким, вероятно, что люди с очень высоким доходом достаточно внимательно относятся к финансовым вопросам, а людей с низким доходом более тщательно проверяет банк, поскольку риски проблем с выплатами в этом случае возрастают. У людей с доходом средним, выше среднего и высоким наблюдается достаточно схожий процент по невыплатам.

**3.4 Влияние цели кредита на его возврат в срок**

In [28]:
purp = pd.pivot_table(df, index = ['purpose_category'], values='debt', aggfunc = ('mean', lambda X:X.count()))

purp['mean'] = purp['mean'].apply(perc)
purp = purp.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее по задолжностям"})
purp.sort_values(by='Количество записей', ascending=False)

,Количество записей,Среднее по задолжностям
purpose_category,,
недвижимость,10811.0,7.23%
автомобиль,4306.0,9.36%
образование,4013.0,9.22%
свадьба,2324.0,8.00%


Взаимосвязь между выплатой в срок кредита и целью не совсем очевидна, попробуем посмотреть не только на цель, но на цель в совокупности с возрастом заемщика.

In [29]:
purp_age = pd.pivot_table(df, index = ['purpose_category', 'age_category'], values='debt', aggfunc = ('mean', lambda X:X.count()))

purp_age['mean'] = purp_age['mean'].apply(perc)
purp_age = purp_age.rename(columns={"<lambda_0>": "Количество записей", "mean": "Среднее по задолжностям"})
purp_age.sort_values(by='Количество записей', ascending=False)

,,Количество записей,Среднее по задолжностям
purpose_category,age_category,,
недвижимость,взрослые,8844.0,6.68%
автомобиль,взрослые,3598.0,8.78%
образование,взрослые,3276.0,8.46%
недвижимость,молодые,1967.0,9.71%
свадьба,взрослые,1918.0,7.61%
образование,молодые,737.0,12.62%
автомобиль,молодые,708.0,12.29%
свадьба,молодые,406.0,9.85%


**Вывод**

Теперь видно, что у категории взрослые людей процент невыплаты кредита гораздо меньше, чем у молодых. Это скорее всего связано с тем, что доход молодых обычно меньше, чем у категории взрослых. При этом категория взрослых чаще всего не выплачивает в срок кредит за автомобиль. Категория молодых заемщиков в целом показывает большие проценты по невыплатам. Чаще всего категория молодых имеет задолженности по кредитам с целью оплаты образования и покупки автомобиля.

## Общий вывод

- Семейный статус влияет на выплату кредита вовремя, заемщики, в разводе или вдовцы выплачивают кредит в срок чаще, чем остальные категории, при этом состоящие в браке выплачивают кредит вовремя чаще, чем люди, не состоящие ни в каких отношениях.
- Бездетные чаще выплачивают в срок кредит, нежели люди с детьми. При этом люди с тремя детьми выплачивают кредит вовремя чаще,чем бездетные.
- Люди с низким и с очень высоким доходом чаще выплачивают кредит в срок.
- Если говорить о целях, то многое зависит от возраста заемщика, так, молодые не выплачивают кредит в срок в среднем чаще, чем взрослые. При этом чаще всего кредит не выплачен в срок, когда целью является образование или покупка автомобиля, а заемщик относится к молодой категории.

Таким образом, выгодным клиентом окажется заемщик:
1. Состоящий в браке
2. Без детей
3. С очень высоким уровнем дохода
4. Из категории взрослых( старше 30 лет).